In [18]:
import pandas as pd 
import duckdb

In [19]:
conn = duckdb.connect('../data/gymBuddy_db.db')

In [20]:
conn.sql("SHOW TABLES")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ workout          │
│ workout_analysis │
└──────────────────┘

In [21]:
conn.sql("Select max(id) from workout")

┌─────────┐
│ max(id) │
│  int32  │
├─────────┤
│     123 │
└─────────┘

In [22]:
conn.sql("select * from workout where id =123")

┌───────┬──────────────┬────────────────────────────┬──────────┬─────────────────┬───────────────────────┬───────────┐
│  ID   │ workout_name │      timestamp_start       │ rep_goal │ strictness_crit │ strictness_definition │ left_side │
│ int32 │   varchar    │         timestamp          │  int32   │     varchar     │        double         │  boolean  │
├───────┼──────────────┼────────────────────────────┼──────────┼─────────────────┼───────────────────────┼───────────┤
│   123 │ push-ups     │ 2025-06-12 15:11:09.055609 │        1 │ loose           │                  15.0 │ true      │
└───────┴──────────────┴────────────────────────────┴──────────┴─────────────────┴───────────────────────┴───────────┘

In [23]:
conn.sql("SELECT * FROM workout_analysis where workout_id = 123")

┌────────────┬───────┬───────────┬───────────────┬───────────┬─────────┬─────────────┬───────────────────────────────────────────────────────────────────┬───────────────────────┐
│ workout_id │ frame │ timestamp │ series_number │ rep_count │  down   │ form_issues │                            angles_data                            │  ldmrks_of_interest   │
│   int32    │ int32 │ timestamp │     int32     │   int32   │ boolean │   varchar   │ struct("name" varchar, "value" double, joint_indices integer[])[] │ map(varchar, integer) │
├────────────┴───────┴───────────┴───────────────┴───────────┴─────────┴─────────────┴───────────────────────────────────────────────────────────────────┴───────────────────────┤
│                                                                                     0 rows                                                                                     │
└────────────────────────────────────────────────────────────────────────────────────────────────────────

In [19]:
import datetime

In [20]:
array_insert = [{'id': 121, 'frame': 162, 'time': datetime.datetime(2025, 6, 12, 14, 6, 40, 900220), 'rep_count': 0, 'down': True, 'form_issues': 'knees on floor', 'angles_data': '[{"name": "elbow", "value": 129.53147363343317, "joint_indices": [15, 13, 11]}, {"name": "body", "value": 177.76583332780172, "joint_indices": [11, 23, 27]}]', 'ldmrks_keys': ['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'], 'ldmrks_values': [11, 13, 15, 23, 27, 25, 31]}, {'id': 121, 'frame': 163, 'time': datetime.datetime(2025, 6, 12, 14, 6, 41, 22448), 'rep_count': 0, 'down': True, 'form_issues': 'Good form! (angle: 165.9degrees)', 'angles_data': '[{"name": "elbow", "value": 137.23768234411716, "joint_indices": [15, 13, 11]}, {"name": "body", "value": 165.9001494314959, "joint_indices": [11, 23, 27]}]', 'ldmrks_keys': ['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'], 'ldmrks_values': [11, 13, 15, 23, 27, 25, 31]}, {'id': 121, 'frame': 164, 'time': datetime.datetime(2025, 6, 12, 14, 6, 41, 136119), 'rep_count': 1, 'down': False, 'form_issues': 'Good form! (angle: 162.4degrees)', 'angles_data': '[{"name": "elbow", "value": 154.16380858568002, "joint_indices": [15, 13, 11]}, {"name": "body", "value": 162.3936643031923, "joint_indices": [11, 23, 27]}]', 'ldmrks_keys': ['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'], 'ldmrks_values': [11, 13, 15, 23, 27, 25, 31]}]

In [21]:
insert_db = pd.DataFrame(array_insert)

In [75]:
data_to_insert = insert_db.copy()

In [76]:
conn.sql("SHOW workout_analysis")

┌────────────────────┬───────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name     │                            column_type                            │  null   │   key   │ default │  extra  │
│      varchar       │                              varchar                              │ varchar │ varchar │ varchar │ varchar │
├────────────────────┼───────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ workout_id         │ INTEGER                                                           │ NO      │ PRI     │ NULL    │ NULL    │
│ frame              │ INTEGER                                                           │ NO      │ NULL    │ NULL    │ NULL    │
│ timestamp          │ TIMESTAMP                                                         │ NO      │ PRI     │ NULL    │ NULL    │
│ series_number      │ INTEGER                                                     

In [77]:
data_to_insert.rename(columns={'id': 'workout_id','time':'timestamp'}, inplace=True)

In [82]:
{'key': data_to_insert['ldmrks_keys'].values, 'value': data_to_insert['ldmrks_values'].values}

{'key': array([list(['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes']),
        list(['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes']),
        list(['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'])],
       dtype=object),
 'value': array([list([11, 13, 15, 23, 27, 25, 31]),
        list([11, 13, 15, 23, 27, 25, 31]),
        list([11, 13, 15, 23, 27, 25, 31])], dtype=object)}

In [86]:
data_to_insert['ldmrks_of_interest']= data_to_insert.apply(lambda row:{'key': row['ldmrks_keys'], 'value': row['ldmrks_values']}, axis=1)


In [87]:
data_to_insert['ldmrks_of_interest']

0    {'key': ['shoulder', 'elbow', 'wrist', 'hip', ...
1    {'key': ['shoulder', 'elbow', 'wrist', 'hip', ...
2    {'key': ['shoulder', 'elbow', 'wrist', 'hip', ...
Name: ldmrks_of_interest, dtype: object

In [88]:
data_to_insert

,workout_id,frame,timestamp,rep_count,down,form_issues,angles_data,ldmrks_keys,ldmrks_values,ldmrks_of_interest
0,121,162,2025-06-12 14:06:40.900220,0,True,knees on floor,"[{""name"": ""elbow"", ""value"": 129.53147363343317...","[shoulder, elbow, wrist, hip, ankle, knee, toes]","[11, 13, 15, 23, 27, 25, 31]","{'key': ['shoulder', 'elbow', 'wrist', 'hip', ..."
1,121,163,2025-06-12 14:06:41.022448,0,True,Good form! (angle: 165.9degrees),"[{""name"": ""elbow"", ""value"": 137.23768234411716...","[shoulder, elbow, wrist, hip, ankle, knee, toes]","[11, 13, 15, 23, 27, 25, 31]","{'key': ['shoulder', 'elbow', 'wrist', 'hip', ..."
2,121,164,2025-06-12 14:06:41.136119,1,False,Good form! (angle: 162.4degrees),"[{""name"": ""elbow"", ""value"": 154.16380858568002...","[shoulder, elbow, wrist, hip, ankle, knee, toes]","[11, 13, 15, 23, 27, 25, 31]","{'key': ['shoulder', 'elbow', 'wrist', 'hip', ..."


In [89]:
data_to_insert.drop(columns=['ldmrks_keys', 'ldmrks_values'], inplace=True)

In [90]:
conn.sql("INSERT INTO workout_analysis BY NAME SELECT * FROM data_to_insert")

In [109]:
conn.sql("SELECT * FROM workout_analysis where workout_id = 122")

┌────────────┬───────┬───────────┬───────────────┬───────────┬─────────┬─────────────┬───────────────────────────────────────────────────────────────────┬───────────────────────┐
│ workout_id │ frame │ timestamp │ series_number │ rep_count │  down   │ form_issues │                            angles_data                            │  ldmrks_of_interest   │
│   int32    │ int32 │ timestamp │     int32     │   int32   │ boolean │   varchar   │ struct("name" varchar, "value" double, joint_indices integer[])[] │ map(varchar, integer) │
├────────────┴───────┴───────────┴───────────────┴───────────┴─────────┴─────────────┴───────────────────────────────────────────────────────────────────┴───────────────────────┤
│                                                                                     0 rows                                                                                     │
└────────────────────────────────────────────────────────────────────────────────────────────────────────

In [24]:
conn.close()